In [3]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [8]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', 2500)        # Let the display adjust to the window


In [5]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
# source_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean_stocks_etfs.parquet'
source_path = Path(DEST_DIR) / f'df_OHLCV_clean_stocks_etfs.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_perf_ratios_stocks_etfs.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-25
DEST_DIR: ..\data

source_path: ..\data\df_OHLCV_clean_stocks_etfs.parquet
dest_path: ..\data\2025-04-25_df_perf_ratios_stocks_etfs.parquet


In [6]:
import pandas as pd

# Get tickers from df_finviz_stocks_etfs.pkl file
df_finviz = pd.read_parquet(f'..\data\{date_str}_df_finviz_stocks_etfs.parquet')
tickers = df_finviz.index.to_list()

In [9]:
print(df_finviz) 

        No.                                     Company               Index                  Sector                        Industry Country Exchange  Market Cap, M    P/E  Fwd P/E   PEG    P/S    P/B    P/C  P/FCF  Book/sh  Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %    EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %  Sales, M  Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M  Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M  ROA %   ROE %  ROI %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %  Beta    ATR  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %  50D High %  50D Low %  52W High %  52W Low %        52W Range  All-Time High %  All-Time Low %    RSI  Earnings    IP

In [38]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
A      2025-05-09   106.9300
       2025-05-08   108.7000
       2025-05-07   107.5200
       2025-05-06   105.2400
       2025-05-05   108.3700
...                      ...
ZWS    2024-02-07    32.7917
       2024-02-06    31.4780
       2024-02-05    30.8064
       2024-02-02    30.4805
       2024-02-01    29.8878

[492536 rows x 1 columns]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 492536 entries, ('A', Timestamp('2025-05-09 00:00:00')) to ('ZWS', Timestamp('2024-02-01 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  492536 non-null  float64
dtypes: float64(1)
memory usage: 5.7+ MB


#### Add date_str arg to select_tickers_data

In [ ]:
import pandas as pd

# def select_tickers_data(df, tickers):
#   """
#   Selects data for a list of tickers from a DataFrame with a MultiIndex
#   where the first level is the ticker and the second level is the date.

#   Args:
#     df (pd.DataFrame): The input DataFrame with a MultiIndex.
#     tickers (list): A list of ticker symbols to select.

#   Returns:
#     pd.DataFrame: A DataFrame containing only the data for the specified tickers.
#                   Rows for tickers not found will not be included.
#   """
#   valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

#   if not valid_tickers:
#     print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
#     return pd.DataFrame()

#   try:
#     ticker_df = df.loc[valid_tickers]
#     return ticker_df
#   except KeyError as e:
#     print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
#     return pd.DataFrame()


# # Example usage:
# # ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
# selected_data = select_tickers_data(df, tickers)
# print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-05-09   198.5300
       2025-05-08   197.4900
       2025-05-07   196.2500
       2025-05-06   198.5100
       2025-05-05   198.8900
...                      ...
FELG   2024-02-07    28.1413
       2024-02-06    27.7735
       2024-02-05    27.7537
       2024-02-02    27.7238
       2024-02-01    27.1423

[484561 rows x 1 columns]


In [ ]:
# import pandas as pd

# def select_tickers_data(df, tickers, end_date=None):
#   """
#   Selects data for a list of tickers from a DataFrame with a MultiIndex
#   where the first level is the ticker and the second level is the date.
#   Filters data to include only dates up to and including the specified end_date.

#   Args:
#     df (pd.DataFrame): The input DataFrame with a MultiIndex (ticker, date).
#     tickers (list): A list of ticker symbols to select.
#     end_date (str or pd.Timestamp, optional): The last date to include in the
#                                              returned data. If None, the last date
#                                              in the entire input DataFrame will be used.
#                                              Dates more recent than end_date are excluded.

#   Returns:
#     pd.DataFrame: A DataFrame containing data for the specified tickers
#                   up to the end_date. Rows for tickers not found or
#                   dates outside the range will not be included.
#   """
#   if not isinstance(df.index, pd.MultiIndex) or len(df.index.levels) < 2:
#     print("DataFrame does not have the expected MultiIndex (ticker, date). Returning an empty DataFrame.")
#     return pd.DataFrame()

#   # Ensure the date level of the index is datetime-like for proper comparison and slicing
#   # This assumes the second level (index 1) is the date level.
#   # We try to convert, but if it's already datetime, it's a no-op.
#   # If it can't be converted, it might lead to errors later or unexpected behavior.
#   try:
#       # Create a new index with the date level converted, then assign back if levels are named
#       # If levels are not named, we have to be careful.
#       # A safer way if levels might not be datetime is to check first or handle errors during slicing.
#       # For now, let's assume it's mostly okay or will be caught by Timestamp conversion later.
#       # It's better if the df comes with date level already as datetime.
#       pass # Let's assume for now the date level is appropriate or will be handled by Timestamp conversion.
#   except Exception as e:
#       print(f"Warning: Could not ensure date level of index is datetime: {e}")


#   # Filter out tickers that do not exist in the DataFrame's first level index
#   all_df_tickers = df.index.get_level_values(0).unique()
#   valid_tickers = [t for t in tickers if t in all_df_tickers]

#   if not valid_tickers:
#     print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
#     return pd.DataFrame()

#   # Determine the actual end_date to use for filtering
#   if end_date is None:
#     if df.empty or df.index.get_level_values(1).empty:
#         print("DataFrame is empty or has no dates to determine a default end_date. Returning empty DataFrame.")
#         return pd.DataFrame()
#     # Default to the last date in the entire DataFrame's second level index
#     actual_end_date = df.index.get_level_values(1).max()
#   else:
#     try:
#       # Convert provided end_date to Timestamp for consistent comparison and slicing
#       actual_end_date = pd.Timestamp(end_date)
#     except ValueError:
#       print(f"Warning: Invalid end_date format '{end_date}'. Using the latest date in the DataFrame instead.")
#       if df.empty or df.index.get_level_values(1).empty:
#           print("DataFrame is empty or has no dates to determine a fallback end_date. Returning empty DataFrame.")
#           return pd.DataFrame()
#       actual_end_date = df.index.get_level_values(1).max()

#   try:
#     # Using .loc with a tuple for MultiIndex: (level0_selection, level1_selection)
#     # slice(None, actual_end_date) selects all dates up to and including actual_end_date
#     # Need to ensure the index is sorted for efficient slicing, especially the date level
#     if not df.index.is_monotonic_increasing:
#         # print("Warning: DataFrame index is not sorted. Sorting for optimal performance.") # Optional warning
#         df_sorted = df.sort_index() # Sort by all levels
#     else:
#         df_sorted = df

#     # Perform the selection
#     # The .loc accessor can handle a list for the first level and a slice for the second level
#     # within a tuple: df.loc[(list_of_tickers, date_slice), :]
#     # The : at the end is for all columns, which is implicit if omitted.
#     ticker_df = df_sorted.loc[(valid_tickers, slice(None, actual_end_date))]

#     return ticker_df
#   except KeyError:
#     # This might happen if, after date slicing, some valid_tickers have no data left.
#     # Or if there's an issue with how .loc interprets the tuple with an empty date slice.
#     # A more robust way is to filter dates first on a broader selection, then tickers,
#     # or handle the empty result from the slice gracefully.
#     # print(f"KeyError during data selection (possibly no data for specified tickers up to end_date): {e}")
#     # If the slice results in no valid dates for any of the tickers, an empty DataFrame is often returned.
#     # If valid_tickers contains items not in df.index.levels[0] (should be caught by valid_tickers check),
#     # or if the date slice is completely out of bounds for *all* selected tickers.
#     # For robustness, return an empty DataFrame.
#     return pd.DataFrame()
#   except Exception as e:
#     print(f"An unexpected error occurred during data selection: {e}")
#     return pd.DataFrame()

# # # --- Example Usage ---
# # if __name__ == '__main__':
# #   # Create a sample DataFrame
# #   data = {
# #       ('AAPL', '2023-01-01'): {'price': 150, 'volume': 1000},
# #       ('AAPL', '2023-01-02'): {'price': 152, 'volume': 1200},
# #       ('AAPL', '2023-01-03'): {'price': 151, 'volume': 1100},
# #       ('AAPL', '2023-01-04'): {'price': 155, 'volume': 1300},
# #       ('MSFT', '2023-01-01'): {'price': 250, 'volume': 800},
# #       ('MSFT', '2023-01-02'): {'price': 255, 'volume': 900},
# #       ('MSFT', '2023-01-03'): {'price': 253, 'volume': 850},
# #       ('GOOG', '2023-01-05'): {'price': 100, 'volume': 2000} # Date later than others
# #   }
# #   df = pd.DataFrame.from_dict(data, orient='index')
# #   df.index = pd.MultiIndex.from_tuples(df.index, names=['ticker', 'date'])
# #   df.index = df.index.set_levels(pd.to_datetime(df.index.levels[1]), level='date') # Convert date level to datetime
# #   df = df.sort_index() # Important for performance and correctness of slicing

# #   print("Original DataFrame:")
# #   print(df)
# #   print("\n" + "="*30 + "\n")

# #   # Test case 1: Select specific tickers with an end_date
# #   print("Test Case 1: AAPL, MSFT, end_date='2023-01-02'")
# #   selected_data_1 = select_tickers_data(df.copy(), ['AAPL', 'MSFT'], end_date='2023-01-02')
# #   print(selected_data_1)
# #   print("\n" + "="*30 + "\n")

# #   # Test case 2: Select tickers, end_date is None (should use max date in df: 2023-01-05)
# #   print("Test Case 2: AAPL, GOOG, end_date=None")
# #   selected_data_2 = select_tickers_data(df.copy(), ['AAPL', 'GOOG'])
# #   print(selected_data_2) # AAPL up to 01-04, GOOG up to 01-05
# #   print("\n" + "="*30 + "\n")

# #   # Test case 3: Ticker not found
# #   print("Test Case 3: XYZ, end_date='2023-01-03'")
# #   selected_data_3 = select_tickers_data(df.copy(), ['XYZ'], end_date='2023-01-03')
# #   print(selected_data_3)
# #   print("\n" + "="*30 + "\n")

# #   # Test case 4: End_date before any data for a ticker
# #   print("Test Case 4: AAPL, end_date='2022-12-31'")
# #   selected_data_4 = select_tickers_data(df.copy(), ['AAPL'], end_date='2022-12-31')
# #   print(selected_data_4)
# #   print("\n" + "="*30 + "\n")

# #   # Test case 5: End_date with invalid format
# #   print("Test Case 5: MSFT, end_date='INVALID-DATE'")
# #   selected_data_5 = select_tickers_data(df.copy(), ['MSFT'], end_date='INVALID-DATE')
# #   print(selected_data_5) # Should use max date of MSFT (2023-01-03) due to fallback
# #   print("\n" + "="*30 + "\n")

# #   # Test case 6: Empty tickers list
# #   print("Test Case 6: Empty tickers list")
# #   selected_data_6 = select_tickers_data(df.copy(), [], end_date='2023-01-03')
# #   print(selected_data_6)
# #   print("\n" + "="*30 + "\n")

# #   # Test case 7: Empty input DataFrame
# #   print("Test Case 7: Empty input DataFrame")
# #   empty_df = pd.DataFrame(columns=['price', 'volume'],
# #                           index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=['ticker', 'date']))
# #   selected_data_7 = select_tickers_data(empty_df, ['AAPL'], end_date='2023-01-03')
# #   print(selected_data_7)
# #   print("\n" + "="*30 + "\n")

# #   # Test case 8: end_date after all data for selected tickers but before max date in df
# #   print("Test Case 8: MSFT, end_date='2023-01-04'")
# #   # MSFT data only up to 2023-01-03. Max date in df is 2023-01-05 (GOOG).
# #   selected_data_8 = select_tickers_data(df.copy(), ['MSFT'], end_date='2023-01-04')
# #   print(selected_data_8) # Should show all MSFT data



In [ ]:
import pandas as pd

def select_tickers_data(df, tickers, end_date=None):
    """
    Selects data for a list of tickers from a DataFrame with a MultiIndex
    where the first level is the ticker and the second level is the date.
    Optionally filters the data to include only dates up to and including end_date.

    Args:
        df (pd.DataFrame): The input DataFrame with a MultiIndex
                           where the first level is the ticker and the second
                           level is the date.
        tickers (list): A list of ticker symbols to select.
        end_date (str or pd.Timestamp, optional): The last date to include
                                                  in the returned DataFrame.
                                                  If None (default), all dates are included.
                                                  Can be a string that pandas can parse as a date
                                                  or a pandas Timestamp object.

    Returns:
        pd.DataFrame: A DataFrame containing only the data for the specified tickers
                      and dates up to the end_date (if provided).
                      Rows for tickers not found will not be included.
    """
    valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

    if not valid_tickers:
        print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
        return pd.DataFrame()

    try:
        ticker_df = df.loc[valid_tickers]
        if end_date is not None:
            if isinstance(end_date, str):
                end_date = pd.to_datetime(end_date)
            # Ensure the second level of the MultiIndex is datetime objects for comparison
            if not isinstance(ticker_df.index.get_level_values(1)[0], pd.Timestamp):
                ticker_df.index = ticker_df.index.set_levels(pd.to_datetime(ticker_df.index.get_level_values(1)), level=1)
            ticker_df = ticker_df[ticker_df.index.get_level_values(1) <= end_date]
        return ticker_df
    except KeyError as e:
        print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
        return pd.DataFrame()

In [43]:
selected_data = select_tickers_data(df, tickers, end_date=date_str)
print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-04-25   209.2800
       2025-04-24   208.3700
       2025-04-23   204.6000
       2025-04-22   199.7400
       2025-04-21   193.1600
...                      ...
FELG   2024-02-07    28.1413
       2024-02-06    27.7735
       2024-02-05    27.7537
       2024-02-02    27.7238
       2024-02-01    27.1423

[469371 rows x 1 columns]


In [45]:
selected_data.loc["A"]

,Adj Close
Date,
2025-04-25,106.280
2025-04-24,107.020
2025-04-23,105.000
2025-04-22,102.810
2025-04-21,100.260
...,...
2024-02-07,131.595
2024-02-06,134.638
2024-02-05,131.625


In [51]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [52]:
print(DAYS_RATIO)
print(date_str)

[3, 5, 10, 15, 30, 60, 120, 250]
2025-04-25


In [53]:
print(len(list_dfs[0].loc["A"]))
print(list_dfs[0].loc["A"])


3
            Adj Close
Date                 
2025-04-23     105.00
2025-04-24     107.02
2025-04-25     106.28


In [54]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
       Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A       5.153101   19.052913  2.697368  15.793771   65.559711  9.259747    3.204522     5.896718   1.619189    1.202716     1.867554   1.226958   -2.258456    -2.893618   0.672376   -3.871692    -4.635103   0.500579    -1.241176     -1.661680    0.804164    -0.846579     -1.118455    0.859515
AA      3.723058   11.141484  1.992562  10.880600   28.880362  4.638584    1.956660     2.954931   1.380855    1.063227     1.742029   1.200455   -2.404414    -3.160253   0.660904   -1.736037    -2.276252   0.738186    -1.412166     -1.848542    0.780054    -0.481224     -0.666162    0.922277
AAL    30.773939         inf       NaN  21.440824         inf       NaN    0.

In [55]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1518 entries, A to ZWS
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sharpe 3d     1518 non-null   float64
 1   Sortino 3d    1518 non-null   float64
 2   Omega 3d      800 non-null    float64
 3   Sharpe 5d     1518 non-null   float64
 4   Sortino 5d    1518 non-null   float64
 5   Omega 5d      930 non-null    float64
 6   Sharpe 10d    1518 non-null   float64
 7   Sortino 10d   1518 non-null   float64
 8   Omega 10d     1515 non-null   float64
 9   Sharpe 15d    1518 non-null   float64
 10  Sortino 15d   1518 non-null   float64
 11  Omega 15d     1518 non-null   float64
 12  Sharpe 30d    1518 non-null   float64
 13  Sortino 30d   1518 non-null   float64
 14  Omega 30d     1518 non-null   float64
 15  Sharpe 60d    1518 non-null   float64
 16  Sortino 60d   1518 non-null   float64
 17  Omega 60d     1518 non-null   float64
 18  Sharpe 120d   1518 non-null   floa

In [56]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            # NaN is from Omega ratio where daily loss series are all zero
            # +Inf is from Sharpe ratio where daily negative excess return series are all zero
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            # -Inf is from Sharpe ratio where daily excess return series are the same, therefore standard deviation is zero
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sortino 3d
  Inf: ['AAL', 'AAON', 'AAPL', 'ABBV', 'ABEV', 'ABNB', 'ACIW', 'ACN', 'ACWI', 'ACWX', 'ADBE', 'AEG', 'AER', 'AFRM', 'AGG', 'AGNC', 'AKAM', 'ALC', 'ALNY', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'AN', 'ANET', 'ANSS', 'APA', 'APG', 'APO', 'APP', 'APTV', 'AR', 'ARCC', 'ARES', 'ARGX', 'ARKB', 'ARKK', 'ARM', 'AS', 'ASML', 'ASND', 'ASR', 'ASX', 'ATI', 'AUR', 'AVDE', 'AVDV', 'AVGO', 'AVUS', 'AWI', 'AXON', 'AZN', 'BA', 'BABA', 'BAC', 'BAH', 'BALL', 'BAM', 'BAP', 'BBEU', 'BBIN', 'BBIO', 'BBJP', 'BBUS', 'BBVA', 'BBY', 'BCH', 'BDX', 'BEP', 'BG', 'BIDU', 'BILS', 'BINC', 'BIRK', 'BITB', 'BIV', 'BKNG', 'BKR', 'BLDR', 'BLV', 'BMRN', 'BN', 'BND', 'BNS', 'BNT', 'BOKF', 'BOND', 'BP', 'BPMC', 'BRBR', 'BRFS', 'BRKR', 'BROS', 'BSAC', 'BSCQ', 'BSCR', 'BSV', 'BSX', 'BUFR', 'BURL', 'BWXT', 'BXSL', 'C', 'CAE', 'CAH', 'CALF', 'CBOE', 'CCJ', 'CDNS', 'CEG', 'CF', 'CFLT', 'CG',

In [57]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'AAL'

#### Use Cell below to check the inf, -inf and NaN replacement

In [58]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       30.773939
Sortino 3d            inf
Omega 3d              NaN
Sharpe 5d       21.440824
Sortino 5d            inf
Omega 5d              NaN
Sharpe 10d       0.627020
Sortino 10d      0.870148
Omega 10d        1.105977
Sharpe 15d       0.959973
Sortino 15d      1.686722
Omega 15d        1.221849
Sharpe 30d      -0.564022
Sortino 30d     -0.880908
Omega 30d        0.893880
Sharpe 60d      -2.831226
Sortino 60d     -3.910800
Omega 60d        0.571980
Sharpe 120d     -0.808785
Sortino 120d    -1.219591
Omega 120d       0.848549
Sharpe 250d     -0.487410
Sortino 250d    -0.718294
Omega 250d       0.907541
Name: AAL, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d          30.773939
Sortino 3d       3802.268850
Omega 3d          339.733032
Sharpe 5d          21.440824
Sortino 5d      15666.484213
Omega 5d         1974.791483
Sharpe 10d          0.627020
Sortino 10d         0.870148
Omega 10d           1.105977
Sharpe 15d          0.959973
Sortino 15d         1.686722
Omega 15d           1.221849
Sharpe 30d         -0.564022
Sortino 30d        -0.880908
Omega 30d           0.893880
Sharpe 60d         -2.831226
Sortino 60d        -3.910800
Omega 60d           0.571980
Sharpe 120d        -0.808785
Sortino 120d       -1.219591
Omega 120d          0.848549
Sharpe 250d        -0.487410
Sortino 250d       -0.718294
Omega 250d          0.907541
Name: AAL, dtype: float64

In [59]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-04-25_df_perf_ratios_stocks_etfs.parquet


In [62]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['AAPL', 'MSFT', 'GOOG', 'NVDA', 'AMZN', 'TSLA', 'META', 'NFLX', 'V', 'JPM', 'GLD', 'MSTR', 'IBIT']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====